# Lab: Even More Linear Regression
## CMSE 381 - Fall 2022
## Lecture 7, Sep 16, 2022

In the last few lectures, we have focused on linear regression, that is, fitting models of the form 
$$
Y =  \beta_0 +  \beta_1 X_1 +  \beta_2 X_2 + \cdots +  \beta_pX_p + \varepsilon
$$
In this lab, we will continue to use two different tools for linear regression. 
- [Scikit learn](https://scikit-learn.org/stable/index.html) is arguably the most used tool for machine learning in python 
- [Statsmodels](https://www.statsmodels.org) provides many of the statisitcial tests we've been learning in class

This lab will cover two ideas: 
- Categorical variables and how to represent them as dummy variables. 
- How to build interaction terms and pass them into your favorite model.

In [1]:
# As always, we start with our favorite standard imports. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import statsmodels.formula.api as smf

# Playing with multi-level variables 

## The wrong way

Ok, we're going to do this incorrectly to start. You pull in the `Auto` data set. You were so proud of yourself for remembering to fix the problems with the `horsepower` column that you conveniently forgot that the column with information about country of origin (`origin`) has a bunch of integers in it, representing:
- 1: `American`
- 2: `European`
- 3: `Japanese`.

In [2]:
Auto_df = pd.read_csv('Auto.csv')
Auto_df = Auto_df.replace('?', np.nan)
Auto_df = Auto_df.dropna()
Auto_df.horsepower = Auto_df.horsepower.astype('int')


Auto_df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin', 'name'],
      dtype='object')

You then go on your merry way building the model 
$$
\texttt{mpg} = \beta_0 + \beta_1 \cdot \texttt{origin}. 
$$

In [3]:
Auto_df.corr()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
mpg,1.000000,-0.777618,-0.805127,-0.778427,-0.832244,0.423329,0.580541,0.565209
cylinders,-0.777618,1.000000,0.950823,0.842983,0.897527,-0.504683,-0.345647,-0.568932
displacement,-0.805127,0.950823,1.000000,0.897257,0.932994,-0.543800,-0.369855,-0.614535
horsepower,-0.778427,0.842983,0.897257,1.000000,0.864538,-0.689196,-0.416361,-0.455171
weight,-0.832244,0.897527,0.932994,0.864538,1.000000,-0.416839,-0.309120,-0.585005
acceleration,0.423329,-0.504683,-0.543800,-0.689196,-0.416839,1.000000,0.290316,0.212746
year,0.580541,-0.345647,-0.369855,-0.416361,-0.309120,0.290316,1.000000,0.181528
origin,0.565209,-0.568932,-0.614535,-0.455171,-0.585005,0.212746,0.181528,1.000000


In [6]:
est = smf.ols('mpg ~ cylinders + displacement + horsepower + weight + acceleration + year + origin', Auto_df).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.821
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     252.4
Date:                Wed, 21 Sep 2022   Prob (F-statistic):          2.04e-139
Time:                        12:24:21   Log-Likelihood:                -1023.5
No. Observations:                 392   AIC:                             2063.
Df Residuals:                     384   BIC:                             2095.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -17.2184      4.644     -3.707      0.000     -26.350      -8.087
cylinders       -0.4934      0.323     -1.526      0.128      -1.129       0.142
displacement     0.0199      0.008      2.647      0.008       0.005       0.035
horsepower      -0.0170      0.014     -1.230      0.220      -0.044       0.010
weight          -0.0065      0.001     -9.929      0.000      -0.008      -0.005
acceleration     0.0806      0.099      0.815      0.415      -0.114       0.275
year             0.7508      0.051     14.729      0.000       0.651       0.851
origin           1.4261      0.278      5.127      0.000       0.879       1.973
==============================================================================
Omnibus:                       31.906   Durbin-Watson:                   1.309
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               53.100
Skew:                           0.529   Prob(JB):                     2.95e-12
Kurtosis:                       4.460   Cond. No.                     8.59e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.59e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [3]:
from sklearn.linear_model import LinearRegression

X = Auto_df.origin.values
X = X.reshape(-1, 1)
y = Auto_df.mpg.values

regr = LinearRegression()

regr.fit(X,y)

print('beta_1 = ', regr.coef_[0])
print('beta_0 = ', regr.intercept_)

beta_1 =  5.4765474801914475
beta_0 =  14.811973615412462


&#9989; **<font color=red>Q:</font>** What does your model predict for each of the three types of cars? 

In [4]:
# Your code here
print("american:", 5.47*1 + 14.81)
print("european:", 5.47*2 + 14.81)
print("japanese:", 5.47*3 + 14.81)

american: 20.28
european: 25.75
japanese: 31.22


&#9989; **<font color=red>Q:</font>** Is it possible for your model to predict that both American and Japanese cars have `mpg` below European cars? 

no

## The right way

Ok, so you figure out your problem and decide to load in your data and fix the `origin` column to have names as entries.

In [5]:
convertOrigin= {1: 'American', 2:'European', 3:'Japanese'}

# This command swaps out each number n for convertOrigin[n], making it one of
# the three strings instead of an integer now.
Auto_df.origin = Auto_df.origin.apply(lambda n: convertOrigin[n])
Auto_df

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,American,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,American,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,American,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,American,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,American,ford torino
...,...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86,2790,15.6,82,American,ford mustang gl
393,44.0,4,97.0,52,2130,24.6,82,European,vw pickup
394,32.0,4,135.0,84,2295,11.6,82,American,dodge rampage
395,28.0,4,120.0,79,2625,18.6,82,American,ford ranger


Below is a quick code that automatically generates our dummy variables. Yay for not having to code that mess ourselves!

In [6]:
origin_dummies_df = pd.get_dummies(Auto_df.origin, prefix='origin')
origin_dummies_df

,origin_American,origin_European,origin_Japanese
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
392,1,0,0
393,0,1,0
394,1,0,0
395,1,0,0


&#9989; **<font color=red>Q:</font>** What is the interpretation of each column in the `origin_dummies_df` data frame?

If there is a one that means that the origin of the car is from that place that is the title of the column

I pass these new dummy variables into my `scikit-learn` linear regression model and get the following coefficients

In [7]:
X = origin_dummies_df.values
y = Auto_df.mpg

regr = LinearRegression()

regr.fit(X,y)

print('Coefs = ', regr.coef_)
print('Intercept = ', regr.intercept_)

Coefs =  [-5.9955451   1.57392668  4.42161842]
Intercept =  26.029014491872697


&#9989; **<font color=red>Q:</font>** Now what does your model predict for each of the three types of cars? 

In [15]:
# Your code here
print("american:", -5.99*1 + 26.02901)
print("european:", 1.57*1 + 26.02901)
print("japanese:", 4.42*1 + 26.02901)

american: 20.039009999999998
european: 27.59901
japanese: 30.44901


### Ooops

&#9989; **<font color=red>Q:</font>** Aw man, I didn't quite do what we said for the dummy variables in class. We talked about having only two dummy variables for a three level variable. Copy my code below here and fix it to have two variables instead of three. 
- Are your coefficients different now?
- Are your predictions for each of the three origins different now? 
- Does it matter which two levels you used for your dummy variables? 

In [16]:
# Your code here
origin_dummies_df_drop = origin_dummies_df.drop(columns = "origin_Japanese")
X = origin_dummies_df_drop.values
y = Auto_df.mpg

regr = LinearRegression()

regr.fit(X,y)

print('Coefs = ', regr.coef_)
print('Intercept = ', regr.intercept_)

Coefs =  [-10.41716352  -2.84769173]
Intercept =  30.4506329113924


## Another right way

Ok, fine, I'll cave, I made you do it the hard way but you got to see how the innards worked, so maybe it's not all bad ;) 

The following code does the same thing as above, but because `statsmodels` has built in tools to handle categorical variables in a data frame, it does the hard work for you. 

In [17]:
est = smf.ols('mpg ~ origin', Auto_df).fit()
est.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,20.0335,0.409,49.025,0.000,19.230,20.837
origin[T.European],7.5695,0.877,8.634,0.000,5.846,9.293
origin[T.Japanese],10.4172,0.828,12.588,0.000,8.790,12.044


&#9989; **<font color=red>Q:</font>** What is the model learned from the above printout? Be specific in terms of your dummy variables. 

the baseline is the american average mpg, so the intercept is the average mpg for americans. All of these are statistcally signifucant as well.

![Stop Icon](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Vienna_Convention_road_sign_B2a.svg/180px-Vienna_Convention_road_sign_B2a.svg.png)

Great, you got to here! Hang out for a bit, there's more lecture before we go on to the next portion. 

# Interaction Terms 

Below is the code I have for generating the tables shown on the slides. 

In [20]:
Advertising_df = pd.read_csv('Advertising.csv', index_col = 0)
Advertising_df.head()

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [21]:
Advertising_df['TVxRadio'] = Advertising_df.TV * Advertising_df.Radio
Advertising_df.head()

,TV,Radio,Newspaper,Sales,TVxRadio
1,230.1,37.8,69.2,22.1,8697.78
2,44.5,39.3,45.1,10.4,1748.85
3,17.2,45.9,69.3,9.3,789.48
4,151.5,41.3,58.5,18.5,6256.95
5,180.8,10.8,58.4,12.9,1952.64


In [22]:
est = smf.ols('Sales ~ TV + Radio + TVxRadio', Advertising_df).fit()
est.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,6.7502,0.248,27.233,0.000,6.261,7.239
TV,0.0191,0.002,12.699,0.000,0.016,0.022
Radio,0.0289,0.009,3.241,0.001,0.011,0.046
TVxRadio,0.0011,5.24e-05,20.727,0.000,0.001,0.001


&#9989; **<font color=red>Do this:</font>** Using the Auto data set, train the model 
$$
\texttt{mpg} = \beta_0 + \beta_1\cdot \texttt{weight} + \beta_2\cdot \texttt{horsepower} + \beta_3\cdot \texttt{weight x horsepower}.
$$
Is the interaction term adding value to the model? 

In [26]:
# Your code here
Auto_df['weightxhorsepower'] = Auto_df.weight * Auto_df.horsepower
est = smf.ols('mpg ~ weight + horsepower + weightxhorsepower', Auto_df).fit()
est.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,63.5579,2.343,27.127,0.000,58.951,68.164
weight,-0.0108,0.001,-13.921,0.000,-0.012,-0.009
horsepower,-0.2508,0.027,-9.195,0.000,-0.304,-0.197
weightxhorsepower,5.355e-05,6.65e-06,8.054,0.000,4.05e-05,6.66e-05


since the p-value is so low it tells us that there is definitely a slope to the interaction term eventhough the slope is so small



-----
### Congratulations, we're done!

Written by Dr. Liz Munch, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.